In [70]:
#Importing libraries
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, MultiLineString
from descartes import PolygonPatch
import matplotlib.pyplot as plt
import folium

In [35]:
# Specify type of data
tags = {'shop': True}

# Download building geometries from OSM
store_gdf = ox.geometries_from_place('Eugene, Oregon, USA', tags)

In [36]:
stores = store_gdf[store_gdf['shop'] == 'supermarket'].reset_index()
stores = stores.dropna(subset=['addr:street'])
stores = stores.dropna(axis=1, how='all')
stores['centroid'] = stores['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))
len(stores)

22

In [37]:
# Specify type of data
tags = {'highway': True}

# Download building geometries from OSM
stops_gdf = ox.geometries_from_place('Eugene, Oregon, USA', tags)

In [38]:
stops = stops_gdf[stops_gdf['highway'] == 'bus_stop'].reset_index()
stops = stops.dropna(axis=1, how='all')
stops = stops[stops.geom_type == 'Point']
stops['centroid'] = stops['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))
len(stops)

206

In [39]:
# Define coordinates
lat_lon = (44.065, -123.0781)

# Define walkable street network 12km
g = ox.graph_from_point(lat_lon, dist=10000, network_type='walk')

In [40]:
# Convert to graph
graph_proj = ox.project_graph(g, to_crs=None)

In [41]:
#Project to same CRS as graph
stores = stores.to_crs(stores.estimate_utm_crs())
stops = stops.to_crs(stops.estimate_utm_crs())
#Turn into centroids
stores['centroid'] = stores['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))

stops['centroid'] = stops['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))

stores.reset_index(drop=True)
stops.reset_index(drop=True)
len(stops)

206

In [42]:
#Make list of centroids and turn into nodes
orig_xy = [stops['centroid'].y.values, stops['centroid'].x.values]
orig_nodes = ox.distance.nearest_nodes(graph_proj, X=orig_xy[1], Y=orig_xy[0], return_dist=False)
target_xy = [stores['centroid'].y.values, stores['centroid'].x.values]
target_nodes = ox.distance.nearest_nodes(graph_proj, X=target_xy[1], Y=target_xy[0], return_dist=False)

In [43]:
#Find all bus stops within 5 minutes walking distance of any grocery store
stops['walking_dist'] = False
for i in range(len(orig_nodes)):
    print(f'{i}')
    routes = []
    lengths = []
    for j in range(len(target_nodes)):
        routes.append(nx.shortest_path(graph_proj, source=orig_nodes[i], target=target_nodes[j], weight='length'))
        lengths.append(nx.shortest_path_length(graph_proj, source=orig_nodes[i], target=target_nodes[j], weight='length'))

    stores['distance_km'] = np.array(lengths) / 1000
    travel_speed_minutes = 4 / 60
    stores['time_minutes'] = stores['distance_km'] / travel_speed_minutes
    if (stores['time_minutes'] <= 5).any() == True:
        stops.at[i, 'walking_dist'] = True

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


In [63]:
#Filter out all stops that do have a store within 5 minutes
stops_by_store = stops[stops['walking_dist'] == True]
stops_by_store = stops_by_store[['element_type', 'osmid', 'geometry', 'name', 'network', 'centroid', 'walking_dist']]
stops_by_store = stops_by_store.reset_index()
stops_by_store = stops_by_store.drop('index', axis=1)
print(f'The number of bus stops with a grocery store within 5 minutes walking is {len(stops_by_store)}')
stops_by_store.head()

The number of bus stops with a grocery store within 5 minutes walking is 32


,element_type,osmid,geometry,name,network,centroid,walking_dist
0,node,1154028388,POINT (493394.749 4877173.024),Hilyard Station (westbound),Emerald Express (EmX),POINT (493394.749 4877173.024),True
1,node,1154028392,POINT (495096.035 4876849.086),Walnut Station Inbound,Emerald Express (EmX),POINT (495096.035 4876849.086),True
2,node,1154028406,POINT (495094.179 4876841.868),Walnut Station Outbound,Emerald Express (EmX),POINT (495094.179 4876841.868),True
3,node,1154028519,POINT (493394.742 4877165.094),Hilyard Station (eastbound),Emerald Express (EmX),POINT (493394.742 4877165.094),True
4,node,2266181107,POINT (492510.325 4882227.966),NaN,NaN,POINT (492510.325 4882227.966),True
